In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import json
%matplotlib inline
from pandas.io.json import json_normalize
import lightgbm

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, KFold


from sklearn.metrics import mean_squared_error
def rmse(y_true, y_pred):
    return round(np.sqrt(mean_squared_error(y_true, y_pred)), 5)


train = pd.read_csv('data/challenge3_train.csv')
test = pd.read_csv('data/challenge3_test.csv')

In [6]:
# Even though catboost handles categorical values, we will have to convert them in order for xgboost to work
from category_encoders import TargetEncoder

train['f12'] = train['f12'].apply(lambda x: x if pd.isnull(x) else np.log10(int(str(x), 16)))

encoder = TargetEncoder()
train['f28'] = encoder.fit_transform(train['f28'], train['target'])

train = pd.get_dummies(train, columns=['f5', 'f15', 'f20'])



/home/mikkel/.cache/pypoetry/virtualenvs/project-J9--KlOd-py3.8/lib/python3.8/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [7]:
# drop id and target
# will drop f12 for now, will handle f12 later
X = train.drop(labels=['id', 'target'], axis=1)
y = train['target']

cat_cols = []#[col for c, col in enumerate(X.columns) \
                        #if not ( np.issubdtype(X.dtypes[c], np.number )  )  ]

print(cat_cols)


# fill missing values with null
X = X.fillna("-9999")
y = y.fillna("-9999")

X = X.apply(pd.to_numeric)
#for col in X.columns:
#    X[X[col]] = pd.to_numeric(X[col])


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)
print(X_train.dtypes)
print(X_train.head)
X_train

[]
f0       float64
f1       float64
f2       float64
f3       float64
f4       float64
f6       float64
f7       float64
f8       float64
f9       float64
f10      float64
f11      float64
f12      float64
f13      float64
f14      float64
f16      float64
f17      float64
f18      float64
f19      float64
f21      float64
f22      float64
f23      float64
f24      float64
f25      float64
f26      float64
f27      float64
f28      float64
f29      float64
f30      float64
f5_F       uint8
f5_M       uint8
f15_A      uint8
f15_B      uint8
f15_C      uint8
f20_A      uint8
f20_B      uint8
f20_C      uint8
f20_D      uint8
f20_E      uint8
f20_F      uint8
f20_G      uint8
f20_H      uint8
f20_I      uint8
f20_J      uint8
f20_K      uint8
f20_L      uint8
f20_M      uint8
f20_N      uint8
f20_O      uint8
f20_P      uint8
f20_Q      uint8
f20_R      uint8
f20_S      uint8
f20_T      uint8
f20_U      uint8
f20_V      uint8
f20_W      uint8
f20_X      uint8
f20_Y      uint8
f20_Z      

,f0,f1,f2,f3,f4,f6,f7,f8,f9,f10,...,f20_Q,f20_R,f20_S,f20_T,f20_U,f20_V,f20_W,f20_X,f20_Y,f20_Z
40063,-9999.0,2.0,0.0,4.0,-9999.0,4.0,-9999.0000,14.2907,0.0,0.0,...,0,0,0,0,0,0,0,1,0,0
7567,0.0,5.0,-9999.0,3.0,-9999.0,2.0,-9999.0000,14.2043,0.0,10.0,...,0,0,0,1,0,0,0,0,0,0
45842,-9999.0,2.0,0.0,-9999.0,912.0,5.0,-7.8826,14.0773,-9999.0,0.0,...,0,0,0,0,0,0,0,0,0,0
48851,-9999.0,3.0,-9999.0,-9999.0,413.0,5.0,-8.8021,14.1852,5.0,34.0,...,0,0,0,0,0,0,0,0,0,0
1215,-9999.0,1.0,0.0,3.0,239.0,4.0,-9999.0000,13.9677,2.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38294,1.0,5.0,-9999.0,3.0,-9999.0,1.0,-9999.0000,14.1860,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
15279,-9999.0,2.0,4.0,2.0,2628.0,5.0,-2.2178,14.2673,-9999.0,40.0,...,0,0,0,0,1,0,0,0,0,0
24046,-9999.0,2.0,0.0,4.0,1504.0,5.0,-3.4708,14.3923,-9999.0,0.0,...,0,0,0,0,0,0,0,0,0,0
18551,-9999.0,-9999.0,0.0,3.0,2917.0,5.0,-9999.0000,14.3348,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# Lightgbm
def lgb_train(X, y, test_X, params):
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    pred_test = 0
    pred_train = 0
    for dev_index, val_index in kf.split(X, y):
        #print(dev_index,val_index)
        train_x, valid_x = X.iloc[dev_index,:], X.iloc[val_index,:]
        train_y, valid_y = y.iloc[dev_index], y.iloc[val_index]
        #X_tr, X_vl = X_train.iloc[tr_idx, :], X_train.iloc[val_idx, :]
        #y_tr, y_vl = y_train.iloc[tr_idx], y_train.iloc[val_idx]
        lgtrain = lightgbm.Dataset(train_x, train_y,categorical_feature=list(cat_cols))
        lgvalid = lightgbm.Dataset(valid_x, valid_y,categorical_feature=list(cat_cols))
        print(lgtrain)
        model = lightgbm.train(params, lgtrain, 2000, valid_sets=[lgvalid], early_stopping_rounds=100, verbose_eval=100)
        pred_test_iter = model.predict(test_X, num_iteration=model.best_iteration)
        pred_test_iter[pred_test_iter<0]=0
        pred_test+=pred_test_iter
        pred_train_iter = model.predict(X, num_iteration=model.best_iteration)
        pred_train_iter[pred_train_iter<0]=0
        pred_train+=pred_train_iter
    pred_test /= 5.
    pred_train  /= 5.
    return pred_test, pred_train

In [9]:
params_lgb = {'objective': 'regression', 
          'metric': 'rmse', 
          'num_leaves': 49, 
          'max_depth': 14, 
          'lambda_l2': 0.01931081461346337, 
          'lambda_l1': 0.007163878762237125, 
          'num_threads': 4, 
          'min_child_samples': 40, 
          'learning_rate': 0.01, 
          'bagging_fraction': 0.7910460446769023, 
          'feature_fraction': 0.5046791892199741, 
          'subsample_freq': 5, 
          'bagging_seed': 42, 
          'verbosity': -1,
        'use_missing': 'true'}

sub_lgb_test, sub_lgb_train = lgb_train(X_train, y_train, X_test, params_lgb)



/home/mikkel/.cache/pypoetry/virtualenvs/project-J9--KlOd-py3.8/lib/python3.8/site-packages/lightgbm/basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 0.409263
[200]	valid_0's rmse: 0.373756
[300]	valid_0's rmse: 0.359402
[400]	valid_0's rmse: 0.35269
[500]	valid_0's rmse: 0.34926
[600]	valid_0's rmse: 0.347542
[700]	valid_0's rmse: 0.346704
[800]	valid_0's rmse: 0.346076
[900]	valid_0's rmse: 0.345637
[1000]	valid_0's rmse: 0.345427
[1100]	valid_0's rmse: 0.345238
[1200]	valid_0's rmse: 0.345057
[1300]	valid_0's rmse: 0.344903
[1400]	valid_0's rmse: 0.344798
[1500]	valid_0's rmse: 0.344652
Early stopping, best iteration is:
[1464]	valid_0's rmse: 0.344606
Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 0.410898
[200]	valid_0's rmse: 0.376259
[300]	valid_0's rmse: 0.361762
[400]	valid_0's rmse: 0.354967
[500]	valid_0's rmse: 0.351472
[600]	valid_0's rmse: 0.349442
[700]	valid_0's rmse: 0.348018
[800]	valid_0's rmse: 0.347185
[900]	valid_0's rmse: 0.346817
[1000]	valid_0's rmse: 0.346483
[1100]	valid_0's rmse: 0.3460

In [12]:
from sklearn.metrics import roc_auc_score

score = roc_auc_score(y_test, sub_lgb_test)
print(score)

0.9162314291092333


In [19]:
import xgboost as xgb

def xgb_train(X, y, test_X, params):
    kf = KFold(n_splits=5, shuffle=True, random_state=2017)
    pred_test_xgb = 0
    pred_train_xgb = 0
    for dev_index, val_index in kf.split(X):
        train_x, valid_x = X.iloc[dev_index,:], X.iloc[val_index,:]
        train_y, valid_y = y.iloc[dev_index], y.iloc[val_index]
        xgb_train_data = xgb.DMatrix(train_x, train_y)
        xgb_val_data = xgb.DMatrix(valid_x, valid_y)
        xgb_submit_data = xgb.DMatrix(test_X)
        xgb_submit_data_train = xgb.DMatrix(X)
        xgb_model = xgb.train(params, xgb_train_data, 
                          num_boost_round=2000, 
                          evals= [(xgb_train_data, 'train'), (xgb_val_data, 'valid')],
                          early_stopping_rounds=100, 
                          verbose_eval=500
                         )
        pred_test = xgb_model.predict(xgb_submit_data, ntree_limit=xgb_model.best_ntree_limit)
        pred_train = xgb_model.predict(xgb_submit_data_train, ntree_limit=xgb_model.best_ntree_limit)
        pred_test[pred_test<0]=0
        pred_train[pred_train<0]=0
        pred_test_xgb += pred_test
        pred_train_xgb += pred_train
    pred_test_xgb /= 5.
    pred_train_xgb /= 5.
    return pred_test_xgb, pred_train_xgb


In [20]:
params_xgb = {
            'objective': 'reg:linear',
            'eval_metric': 'rmse',
            'eta': 0.001,
            'max_depth': 7,
            'gamma': 1.3250360141843498, 
            'min_child_weight': 13.0958516960316, 
            'max_delta_step': 8.88492863796954, 
            'subsample': 0.9864199446951019, 
            'colsample_bytree': 0.8376539278239742,
            'subsample': 0.6,
            'colsample_bytree': 0.8,
            'alpha':0.001,
            "num_leaves" : 40,
            'random_state': 42,
            'silent': True,
            }


In [21]:
sub_xgb_test, sub_xgb_train = xgb_train(X_train, y_train, X_test, params_xgb)

[14:24:31] WARNING: ../src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[14:24:31] WARNING: ../src/learner.cc:516: 
Parameters: { num_leaves, silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-rmse:0.49982	valid-rmse:0.49982
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 100 rounds.
[500]	train-rmse:0.43553	valid-rmse:0.43813
[1000]	train-rmse:0.39895	valid-rmse:0.40394
[1500]	train-rmse:0.37656	valid-rmse:0.38374
[1999]	train-rmse:0.36217	valid-rmse:0.37112
[14:26:24] WARNING: ../src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[14:26:32] WARNING: ../src/objective/regression_obj.cu:17

In [24]:
score = roc_auc_score(y_test, sub_xgb_test)
print(score)

0.8973767089759042


In [27]:
from catboost import CatBoostRegressor

def cat_train(X, y, test_X):
    kf = KFold(n_splits=5, shuffle=True, random_state=2017)
    pred_test_cat = 0
    pred_train_cat = 0
    for dev_index, val_index in kf.split(X):
        train_x, valid_x = X.iloc[dev_index,:], X.iloc[val_index,:]
        train_y, valid_y = y.iloc[dev_index], y.iloc[val_index]
        model = CatBoostRegressor(iterations=1000,
                             learning_rate=0.05,
                             depth=10,
                             eval_metric='RMSE',
                             random_seed = 42,
                             bagging_temperature = 0.2,
                             od_type='Iter',
                             metric_period = 50,
                             od_wait=20)
        model.fit(train_x, train_y, eval_set=(valid_x, valid_y),use_best_model=True,verbose=True, 
                  cat_features= [i for i in range(len(X.columns)) if X.columns[i] in cat_cols])
        pred_test = model.predict(test_X)
        pred_test[pred_test<0]=0
        pred_test_cat += pred_test
        pred_train = model.predict(X)
        pred_train[pred_train<0]=0
        pred_train_cat += pred_train
    pred_test_cat /= 5.
    pred_train_cat /= 5.
    return pred_test_cat, pred_train_cat

In [28]:
sub_cat_test, sub_cat_train = cat_train(X_train, y_train, X_test)

score = roc_auc_score(y_test, sub_cat_test)
print(score)

0:	learn: 0.4873078	test: 0.4887855	best: 0.4887855 (0)	total: 105ms	remaining: 1m 45s
50:	learn: 0.3618786	test: 0.3738022	best: 0.3738022 (50)	total: 2.05s	remaining: 38.2s
100:	learn: 0.3362814	test: 0.3585506	best: 0.3585506 (100)	total: 3.89s	remaining: 34.6s
150:	learn: 0.3194446	test: 0.3538325	best: 0.3538325 (150)	total: 5.74s	remaining: 32.3s
200:	learn: 0.3053412	test: 0.3509498	best: 0.3509498 (200)	total: 7.56s	remaining: 30s
250:	learn: 0.2927056	test: 0.3490182	best: 0.3490182 (250)	total: 9.42s	remaining: 28.1s
300:	learn: 0.2814068	test: 0.3480867	best: 0.3480867 (300)	total: 11.2s	remaining: 26.1s
350:	learn: 0.2703113	test: 0.3474067	best: 0.3473912 (348)	total: 13s	remaining: 24.1s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.3472144634
bestIteration = 376

Shrink model to first 377 iterations.


0:	learn: 0.4876792	test: 0.4873941	best: 0.4873941 (0)	total: 39.9ms	remaining: 39.9s
50:	learn: 0.3616220	test: 0.3753432	best: 0.3753432 (50)	total: 1.94s	remaining: 36s
100:	learn: 0.3353095	test: 0.3602031	best: 0.3602031 (100)	total: 4.02s	remaining: 35.8s
150:	learn: 0.3182662	test: 0.3556377	best: 0.3556243 (149)	total: 5.84s	remaining: 32.9s
200:	learn: 0.3043650	test: 0.3530324	best: 0.3530324 (200)	total: 7.66s	remaining: 30.5s
250:	learn: 0.2915052	test: 0.3515702	best: 0.3515702 (250)	total: 9.46s	remaining: 28.2s
300:	learn: 0.2801411	test: 0.3512098	best: 0.3511706 (296)	total: 11.2s	remaining: 26s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.3506785588
bestIteration = 327

Shrink model to first 328 iterations.


0:	learn: 0.4879163	test: 0.4878404	best: 0.4878404 (0)	total: 39.2ms	remaining: 39.2s
50:	learn: 0.3624736	test: 0.3712391	best: 0.3712391 (50)	total: 1.86s	remaining: 34.7s
100:	learn: 0.3366622	test: 0.3563337	best: 0.3563337 (100)	total: 3.64s	remaining: 32.4s
150:	learn: 0.3199661	test: 0.3516290	best: 0.3516290 (150)	total: 5.41s	remaining: 30.4s
200:	learn: 0.3056477	test: 0.3492098	best: 0.3492098 (200)	total: 7.21s	remaining: 28.7s
250:	learn: 0.2926048	test: 0.3478709	best: 0.3478709 (250)	total: 8.97s	remaining: 26.8s
300:	learn: 0.2815137	test: 0.3467263	best: 0.3467263 (300)	total: 10.7s	remaining: 24.9s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.3464112726
bestIteration = 328

Shrink model to first 329 iterations.


0:	learn: 0.4876329	test: 0.4888278	best: 0.4888278 (0)	total: 37.1ms	remaining: 37s
50:	learn: 0.3611970	test: 0.3753608	best: 0.3753608 (50)	total: 1.86s	remaining: 34.6s
100:	learn: 0.3351533	test: 0.3608430	best: 0.3608430 (100)	total: 3.67s	remaining: 32.7s
150:	learn: 0.3180629	test: 0.3558484	best: 0.3558484 (150)	total: 5.47s	remaining: 30.8s
200:	learn: 0.3032263	test: 0.3534105	best: 0.3534105 (200)	total: 7.26s	remaining: 28.9s
250:	learn: 0.2906804	test: 0.3520311	best: 0.3520033 (249)	total: 9.05s	remaining: 27s
300:	learn: 0.2791841	test: 0.3508453	best: 0.3508453 (300)	total: 10.8s	remaining: 25.1s
350:	learn: 0.2688767	test: 0.3502062	best: 0.3501442 (346)	total: 12.6s	remaining: 23.3s
400:	learn: 0.2587524	test: 0.3496118	best: 0.3496118 (400)	total: 14.4s	remaining: 21.5s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.3492004154
bestIteration = 415

Shrink model to first 416 iterations.


0:	learn: 0.4878655	test: 0.4877049	best: 0.4877049 (0)	total: 38.9ms	remaining: 38.8s
50:	learn: 0.3609925	test: 0.3787653	best: 0.3787653 (50)	total: 1.83s	remaining: 34.1s
100:	learn: 0.3350705	test: 0.3634512	best: 0.3634512 (100)	total: 3.62s	remaining: 32.2s
150:	learn: 0.3176642	test: 0.3587214	best: 0.3587214 (150)	total: 5.4s	remaining: 30.4s
200:	learn: 0.3035271	test: 0.3564895	best: 0.3564693 (198)	total: 7.17s	remaining: 28.5s
250:	learn: 0.2917330	test: 0.3553875	best: 0.3553847 (249)	total: 8.95s	remaining: 26.7s
300:	learn: 0.2814278	test: 0.3547965	best: 0.3547965 (300)	total: 10.7s	remaining: 24.8s
350:	learn: 0.2711899	test: 0.3537363	best: 0.3537363 (350)	total: 12.4s	remaining: 23s
400:	learn: 0.2625030	test: 0.3533101	best: 0.3532685 (392)	total: 14.1s	remaining: 21.1s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.3532684729
bestIteration = 392

Shrink model to first 393 iterations.
0.9110842975749052


****Stacking****

In [29]:
last = pd.DataFrame()
last['lgbm'] = sub_lgb_train
last['xbm'] = sub_xgb_train
last['cat'] = sub_cat_train

last_test = pd.DataFrame()
last_test['lgbm'] = sub_lgb_test
last_test['xbm'] = sub_xgb_test
last_test['cat'] = sub_cat_test

In [31]:
from sklearn.linear_model import Ridge
model = Ridge().fit(last, y_train)
pred = model.predict(last_test)

score = roc_auc_score(y_test, pred)
print(score)

0.8990930399386554
